In [5]:
import os
import fnmatch
import re

def search_string_in_files(directory, search_string):
    matches = []
    pattern = re.compile(search_string)

    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, '*'):
            filepath = os.path.join(root, filename)
            try:
                with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                    for line_number, line in enumerate(file, 1):
                        if pattern.search(line):
                            matches.append({
                                'file': filepath,
                                'line_number': line_number,
                                'line': line.strip()
                            })
            except Exception as e:
                print(f"Could not read file {filepath}: {e}")

    return matches

# Example usage
directory_to_search = 'C:/Users/alm/git/Imswitch_descspim/'
#directory_to_search = 'C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/'
print(directory_to_search)

string_to_search = '_getPower'

results = search_string_in_files(directory_to_search, string_to_search)

for match in results:
    print(f"Found in {match['file']} on line {match['line_number']}: {match['line']}")


C:/Users/alm/git/Imswitch_descspim/
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\lasers\PyCobolt0601LaserManager.py on line 92: def _getPower(self):
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\lasers\__pycache__\PyCobolt0601DPLLaserManager.cpython-310.pyc on line 33: setEnabled,   s   z&PyCobolt0601DPLLaserManager.setEnabledc                 C   s*   t |}| jr| | d S | | d S N)intr   _setModPower_setBasicPowerr$   powerr/   r/   r0  setValue2     z$PyCobolt0601DPLLaserManager.setValuec                 C   sb   |r|   }| j| | j  | jd n|   }| jj|d | jd|  || _d S )NzEntered modulation mode)r9   z@Exited digital modulation mode, constant power mode with power: 	_getPowerr   r   r   r   r   constant_powerr   )r$   activer9   r/   r/   r0   r1   9   s   
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\lasers\

In [1]:
import pycobolt
import time

laser = pycobolt.Cobolt06DPL(port='COM12') # Creates a new Cobolt Laser object.
time.sleep(2)
laser.is_connected()

True

In [2]:
dir(laser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_classify_',
 '_identify_',
 '_timeDiff_',
 'address',
 'analog_modulation',
 'baudrate',
 'clear_fault',
 'connect',
 'constant_current',
 'constant_power',
 'digital_modulation',
 'disconnect',
 'get_current',
 'get_current_setpoint',
 'get_fault',
 'get_mode',
 'get_modualtion_tec_setpoint',
 'get_modulation_current',
 'get_modulation_state',
 'get_modulation_tec',
 'get_ophours',
 'get_power',
 'get_power_setpoint',
 'get_state',
 'interlock',
 'is_connected',
 'is_on',
 'modelnumber',
 'modulation_mode',
 'port',
 'send_cmd',
 'serialnumber',
 'set_current',
 'set_modulation_current_high',

In [5]:
laser.get_state()

'1 - Waiting for key'

In [32]:
laser.set_modulation_power(5)

'OK'

In [1]:
laser.modulation_mode()

NameError: name 'laser' is not defined

In [2]:
laser2 = pycobolt.Cobolt06DPL(port='COM12') # Creates a new Cobolt Laser object.
time.sleep(2)
laser2.is_connected()

True